<a href="https://colab.research.google.com/github/rabiyak19/emotion_detection_cnn/blob/master/emotion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json,load_model
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import *



**Loading** **Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/My Drive/emotion/fer2013.csv')

**Analyzing Data**

In [ ]:
data.info()

In [ ]:
data['emotion'].unique()

0  Anger
1  Disgust
2  Fear
3  Happy
4  Sad
5  Surprise
6  Neutral

In [ ]:
data['Usage'].unique()

In [ ]:
data['Usage'].value_counts()

**Making Training Validation & Test Sets**

In [ ]:
training_data =  data.where(data['Usage']=='Training')

In [ ]:
training_data = training_data.dropna(axis=0)

In [ ]:
training_data

In [ ]:
validation_data = data.iloc[28710:32300]

In [ ]:
validation_data = validation_data.dropna(axis=0)

In [ ]:
validation_data

In [ ]:
test_data = data.where(data['Usage']=='PrivateTest') 

In [ ]:
test_data = test_data.dropna(axis=0)

In [ ]:
test_data

**Preparing Data**

In [ ]:
training_data.to_csv('train.csv',index= False)
validation_data.to_csv('validation.csv', index= False)
test_data.to_csv('test.csv', index= False)

In [ ]:
train_data = pd.read_csv('train.csv')
train_data

In [ ]:
def extract_features(filename):
  X = []
  Y = []
  headers = True
  for line in open(filename):
    if headers:
      headers = False
    else:
      row = line.split(',')
      Y.append(row[0])
      X.append([int(feature) for feature in row[1].split()])
      
  X,Y = np.array(X)/255.0, np.array(Y)

  return (X,Y)

In [ ]:
train_x, train_y = extract_features('train.csv')
validation_x, validation_y = extract_features('validation.csv')
test_x, test_y = extract_features('test.csv')

In [ ]:
train_x = np.array(train_x,'float32')
train_y = np.array(train_y,'float32')
validation_x = np.array(validation_x,'float32')
validation_y = np.array(validation_y,'float32')
test_x = np.array(test_x,'float32')
test_y = np.array(test_y,'float32')

In [ ]:
print(validation_x.shape)
print(train_y.shape)

In [ ]:
classes = len(set(train_y))
print(classes)

Reshaping dimensions for the model. 
48x48 is the size of images.
n is number of images. d is the dimension.

In [ ]:
n_train, d_train = train_x.shape
n_validation, d_validation = validation_x.shape
n_test, d_test = test_x.shape 

In [ ]:
train_x = train_x.reshape(n_train, 48, 48, 1)                 #1 for grayscale images
validation_x = validation_x.reshape(n_validation, 48, 48, 1)
test_x = test_x.reshape(n_test, 48, 48, 1)

In [ ]:
print(train_x.shape)
print(validation_x.shape)
print(test_x.shape)

In [ ]:
train_y = to_categorical(train_y)  
validation_y = to_categorical(validation_y)
test_y = to_categorical(test_y)

In [ ]:
train_y[0]

emotion_analysis will show bar graph for predicted img

In [ ]:
def emotion_analysis(emotions):
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()

In [ ]:
def display_predictions(img_file, model):
  img = image.load_img(img_file, grayscale=True, target_size=(48, 48))
  show_img=image.load_img(img_file, grayscale=False, target_size=(200, 200))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis = 0)
  x /= 255

  custom_img = model.predict(x)
  #print(custom[0])
  emotion_analysis(custom_img[0])

  x = np.array(x, 'float32')
  x = x.reshape([48, 48]);

  plt.gray()
  plt.imshow(show_img)
  plt.show()

  m=0.000000000000000000001
  a=custom[0]
  for i in range(0,len(a)):
      if a[i]>m:
          m=a[i]
          ind=i
          
  print('Expression Prediction:',objects[ind])

**Defining Base Model**

In [ ]:
def model_tuned():
  model = Sequential()
  model.add(Conv2D(32, (3,3), activation='relu',input_shape=(48, 48, 1), padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  #model.add(Dropout(0.5))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
  # model.add(MaxPooling2D((2, 2)))
  # model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
 
  model.add(Flatten())
  model.add(Dropout(0.5))
  model.add(Dense(512, activation='relu'))
  model.add(Dense(7, activation='softmax'))
  my_optimiser = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam')
        
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=my_optimiser)
  model.summary()    
  return model

 

In [ ]:
model1 = model_tuned()

In [ ]:
history = model1.fit(train_x, train_y, batch_size=64, epochs=100, verbose=1, validation_data=(validation_x,validation_y), shuffle=True,)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model1.save('emotion_det_model.h5')

**Visualising** **convnet**

In [ ]:
img_path = '/content/drive/My Drive/emotion/angry1.jpeg'
from keras.preprocessing import image
img = image.load_img(img_path, target_size=(48, 48), grayscale=True)
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.0


In [ ]:
plt.imshow(img_tensor[0])
plt.show()

In [ ]:
from keras import models

layer_outputs = [layer.output for layer in model1.layers[:5]]
activation_model = models.Model(inputs=model1.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(img_tensor)

In [ ]:
first_layer_activation = activations[0]

In [ ]:
print(first_layer_activation.shape)

In [ ]:
plt.matshow(first_layer_activation[0, :, :, 3], cmap='viridis')

In [ ]:
layer_names = []

for layer in model1.layers[:5]:
  layer_names.append(layer.name)

images_per_row = 16

for layer_name, layer_activation in zip(layer_names, activations):
  n_features = layer_activation.shape[-1]
  size = layer_activation.shape[1]
  n_cols = n_features // images_per_row
  display_grid = np.zeros((size * n_cols, images_per_row * size))
  for col in range(n_cols):
    for row in range(images_per_row):
      channel_image = layer_activation[0,:, :,col * images_per_row + row]
      channel_image -= channel_image.mean()
      channel_image /= channel_image.std()
      channel_image *= 64
      channel_image += 128
      channel_image = np.clip(channel_image, 0, 255).astype('uint8')
      display_grid[col * size : (col + 1) * size,
      row * size : (row + 1) * size] = channel_image
  scale = 1. / size
  plt.figure(figsize=(scale * display_grid.shape[1],
  scale * display_grid.shape[0]))
  plt.title(layer_name)
  plt.grid(False)
  plt.imshow(display_grid, aspect='auto', cmap='viridis')